In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from pyfolio.timeseries import perf_stats
%matplotlib inline

In [2]:
allocs = pd.Series([.05, .3, .35, .3, 1], index=['CL', 'ES', 'GC', 'NQ', 'total'])
cash = 100000

In [3]:
df = pd.read_csv('/home/tomek/ib_tools/backtests/backtest_2020-02-23_13-55_20190101_20191031.csv')
del df['sys_time']
del df['exec_ids']
try:
    del df['perm_ids']
    del df['com_reports']
except:
    pass
df['time'] = pd.to_datetime(df['time'])
df.set_index('time', inplace=True)
df['cash'] = df.realizedPNL.cumsum() + cash
df['contract'] = df['contract'].apply(lambda x: x[:-2])
df

,contract,action,amount,price,order_id,perm_id,reason,commission,realizedPNL,com_reports,cash
time,,,,,,,,,,,


In [4]:
df[(df['contract'] == 'CL') & (df['realizedPNL'] == 0)].count()

contract       0
action         0
amount         0
price          0
order_id       0
perm_id        0
reason         0
commission     0
realizedPNL    0
com_reports    0
cash           0
dtype: int64

In [5]:
df[df['contract'] == 'NQ'].loc['20190601':]

,contract,action,amount,price,order_id,perm_id,reason,commission,realizedPNL,com_reports,cash
time,,,,,,,,,,,


In [6]:
df[df['contract'] == 'NQ'].price.plot(figsize=(20,8), grid=True);

TypeError: no numeric data to plot

In [ ]:
df.groupby('contract').realizedPNL.sum()

In [ ]:
df.realizedPNL.sum()

In [ ]:
df.cash.plot(figsize=(20,4), grid=True);

In [ ]:
by_contract = pd.pivot_table(df, values='realizedPNL', index=df.index, columns=['contract'])
by_contract['total'] = by_contract.sum(axis=1)

In [ ]:
# number of trades
trades = df.groupby('contract').action.count()
trades

In [ ]:
# commission paid
df.groupby('contract').commission.sum()

In [ ]:
df.commission.sum()

In [ ]:
# per trade stats
exits = df[df.realizedPNL!=0]
dfs = {}
for contract in exits.contract.unique():
    dfs[contract] = exits[exits.contract == contract]
dfs['total'] = exits
results = pd.DataFrame(columns=['success_rate', 'average_PNL', 'average_gain', 'average_loss','win_ratio', 'EV',
                               'number_of_positions'])
for c, d in dfs.items():
    row = pd.Series()
    row['success_rate'] = round((d[d.realizedPNL > 0]).realizedPNL.count() / d.realizedPNL.count(), 2)
    row['average_PNL'] = round(d.realizedPNL.mean(), 2)
    row['average_gain'] = round((d[d.realizedPNL > 0]).realizedPNL.mean(), 2)
    row['average_loss'] = round((d[d.realizedPNL < 0]).realizedPNL.mean(), 2)
    row['win_ratio'] = round(abs(row['average_gain'] / row['average_loss']), 2)
    row['EV'] = round((row['average_gain'] * row['success_rate']) + (
        row['average_loss'] * (1 - row['success_rate'])), 2)
    row['number_of_positions'] = d.realizedPNL.count()
    results.loc[c] = row
results

In [ ]:
daily = by_contract.resample('B').sum()
daily = daily.cumsum() +  (allocs * cash)
#daily[['CLZ9', 'ESZ9', 'GCZ9', 'NQZ9']].plot(figsize=(20,8), grid=True);

In [ ]:
# instrument correlations
log_returns = np.log(daily.pct_change()[1:] + 1)
log_returns.corr()

In [ ]:
log_returns.cumsum().plot(figsize=(20,8), grid=True);

In [ ]:
returns = daily.pct_change()
returns.iloc[0] = (daily.iloc[0] / (allocs * cash))-1
returns_index = (returns + 1).cumprod()
returns_index.plot(figsize=(20,8), grid=True);

In [ ]:
stats = pd.DataFrame()
for col in returns.columns:
    stats[col] = perf_stats(returns[col])
stats

In [ ]:
# monthly returns in percent
monthly_log_returns = log_returns.resample('M').sum()
monthly_returns = np.exp(monthly_log_returns)-1
round(monthly_returns, 4) * 100

In [ ]:
# total returns by contract in percent
round(np.exp(monthly_log_returns.sum())-1, 2)* 100

In [ ]:
df.sort_values('realizedPNL', ascending=False)[['contract', 'action', 'price', 'realizedPNL']]